In [102]:
import import_ipynb
import sys
import os

sys.path.append(os.path.abspath("../../"))  # Go up one level and into 'data'
import data_synthesis.generate_discrete_data as gdd  # Import function and class

In [103]:
def test(concrete_space, abstract_space, target_abstraction, true_architecture, proposed_architecture, loss_function, specific_args):
    """
    Ideally the concrete space should be sampled randomly and the abstract space should be sampled on the run but perhaps having limitations
    concrete_space: The possible input space of the optimization
    abstract_space: The space containing the candidates to optimize the proposed architecture
    target_abstraction: It is the parameters of the true_architecture
    true_architecture: The way inputs and parameters are related in the real situation
    proposed_architecture: The our model perceives the relation between inputs and parameters
    loss_function: Function to evaluate results
    specific_args: Argment depending on optimizer
    """

In [104]:
import numpy as np

def hill_climb(concrete_space, abstract_space, target_abstraction, true_architecture, proposed_architecture, loss_function, specific_args):
    # Extract hyperparameters
    max_iterations = specific_args.get("max_iterations", 100)
    step_size = specific_args.get("step_size", 0.1)

    # Randomly initialize parameters from the abstract space
    current_params = np.array(abstract_space[np.random.randint(len(abstract_space))])

    # Compute initial loss
    true_outputs = [true_architecture(x, target_abstraction) for x in concrete_space]
    proposed_outputs = [proposed_architecture(x, current_params) for x in concrete_space]
    current_loss = np.mean([loss_function(t, p) for t, p in zip(true_outputs, proposed_outputs)])

    for _ in range(max_iterations):
        # Generate new candidate by adding small random variation
        candidate_params = current_params + np.random.uniform(-step_size, step_size, size=len(current_params))

        # Evaluate new loss
        proposed_outputs = [proposed_architecture(x, candidate_params) for x in concrete_space]
        new_loss = np.mean([loss_function(t, p) for t, p in zip(true_outputs, proposed_outputs)])

        # Accept if the candidate improves the loss
        if new_loss < current_loss:
            current_params, current_loss = candidate_params, new_loss

    return {"best_params": current_params, "best_loss": current_loss}


In [105]:
# Define a sample function to optimize (e.g., linear model)
def true_architecture(x, params):
    a, b = params
    return a * x + b

def proposed_architecture(x, params):
    a, b = params
    return a * x + b + np.random.normal(0, 0.1)  # Adding slight noise to simulate imperfection

# Define a loss function (Absolute Error)
def absolute_error(true, pred):
    return abs(true - pred)

# Define spaces
space_shape = [10]  # 1D space
limits = np.array([[-5, 5]])  # Limits for x-values

# Generate concrete samples instead of a full grid
concrete_space = gdd.ConcreteSpace(space_shape, limits, "monte_carlo")
concrete_samples = concrete_space.generate_random_samples(n=50)

# Define abstract parameter space
abstract_space = [np.array([a, b]) for a in np.linspace(-2, 2, 10) for b in np.linspace(-2, 2, 10)]

# Define true parameters
target_abstraction = abstract_space[0]

# Run optimization
best_solution = hill_climb(
    concrete_samples, abstract_space, target_abstraction,
    true_architecture, proposed_architecture,
    absolute_error, specific_args={"max_iterations": 1000, "step_size": 0.1}
)

# Print best found parameters
print("Best Parameters:", best_solution["best_params"])
print("Best Loss:", best_solution["best_loss"])


Best Parameters: [-2.00191204 -1.98624784]
Best Loss: 0.062276867380932105
